In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="{message}", level="INFO")

1

In [3]:
from ipissa.train.train_adapter import evaluate_model, load_model, load_labels, TrainingConfig, get_choice_ids, select_dilemma_by_values, load_and_process_daily_dilemmas_eval_dataset, process_daily_dilemma_results, proj_root, register_ipissa_peft, setup_adapter, add_adapter_name_to_sd, train_steer_vector
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
import pandas as pd
import gc

In [4]:

results_dir = sorted(( proj_root / "./outputs/adapters/").glob("*"))
for res_dir in results_dir:
    print(res_dir)
    if 'q4' in res_dir.name:
        results_dir = res_dir

adapter_path = results_dir / "adapter_model.safetensors"

/workspace/InnerPiSSA_private/outputs/adapters/all_20251121_212708
/workspace/InnerPiSSA_private/outputs/adapters/attn.down mlp.up_20251122_043556
/workspace/InnerPiSSA_private/outputs/adapters/data_100_20251123_023624
/workspace/InnerPiSSA_private/outputs/adapters/data_2000_20251123_032024
/workspace/InnerPiSSA_private/outputs/adapters/data_200_20251123_024240
/workspace/InnerPiSSA_private/outputs/adapters/data_400_20251123_025042
/workspace/InnerPiSSA_private/outputs/adapters/data_50_20251123_023014
/workspace/InnerPiSSA_private/outputs/adapters/data_800_20251123_030226
/workspace/InnerPiSSA_private/outputs/adapters/gemma3-raw-r128-lr6e-3_20251122_061156
/workspace/InnerPiSSA_private/outputs/adapters/gemma3-raw-r128-lr6e-3_20251122_083454
/workspace/InnerPiSSA_private/outputs/adapters/gemma3-raw-r128-lr6e-3_20251122_113115
/workspace/InnerPiSSA_private/outputs/adapters/gemma3-raw-r128-lr6e-3_20251122_234339
/workspace/InnerPiSSA_private/outputs/adapters/gemma3-raw-r128-lr6e-3_2025112

In [5]:
results_dir

PosixPath('/workspace/InnerPiSSA_private/outputs/adapters/q4b-raw-r8-L5-lr1e-2_20251121_094545')

In [6]:
# load
from ipissa.peft_utils.layer_selection import LayerSelection, compute_layer_selection
import json, cattrs
# now evaluate with prompts
d = json.loads((results_dir / "training_config.json").read_text())
config = cattrs.structure(d, TrainingConfig)
model_id = config.model_name
base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)

config.eval_max_tokens = 256
config.eval_max_dilemmas = 1000
# config.eval_batch_size = 6
# load adapter

# Register InnerPiSSA
register_ipissa_peft()

# Compute layer selection ONCE (single source of truth)
layer_selection = compute_layer_selection(
    base_model,
    depth_start=config.depth_start,
    depth_end=config.depth_end,
    n_depths=config.n_depths,
    loss_depths=config.loss_depths,
    modules=config.modules,
    loss_modules=config.loss_modules,
)

model = setup_adapter(
    base_model, 
    config, 
    target_modules=layer_selection.adapter_regex,
    # init_steering_vecs=init_steering_vecs
)

import safetensors.torch
sd = safetensors.torch.load_file(adapter_path)
sd = add_adapter_name_to_sd(sd, adapter_name='honest', prefix="ipissa_")

# now we need to add in .default ?
r = model.load_state_dict(sd, strict=False)
assert not r.unexpected_keys, f"Unexpected keys in state_dict: {r.unexpected_keys[:5]}, missing_keys {r.missing_keys[::5]}"

# model.model.load_adapter(results_dir)
config

Loading model: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Target modules regex: .*\.(10|15|22|27|33)\..*(gate_proj|gate_proj|gate_proj|gate_proj|gate_proj|up_proj|up_proj|up_proj|up_proj|up_proj)
InnerPiSSA Layer Init: model.layers.10.mlp.gate_proj, r=8, norms W=132.5, res=129.9, Wr=25.7
InnerPiSSA Layer Init: model.layers.10.mlp.up_proj, r=8, norms W=110.1, res=109.5, Wr=11.5
InnerPiSSA Layer Init: model.layers.15.mlp.gate_proj, r=8, norms W=114.9, res=113.3, Wr=18.8
InnerPiSSA Layer Init: model.layers.15.mlp.up_proj, r=8, norms W=114.3, res=113.4, Wr=14.0
InnerPiSSA Layer Init: model.layers.22.mlp.gate_proj, r=8, norms W=114.9, res=113.6, Wr=17.3
InnerPiSSA Layer Init: model.layers.22.mlp.up_proj, r=8, norms W=116.7, res=115.9, Wr=13.6
InnerPiSSA Layer Init: model.layers.27.mlp.gate_proj, r=8, norms W=122.3, res=120.9, Wr=18.1
InnerPiSSA Layer Init: model.layers.27.mlp.up_proj, r=8, norms W=120.4, res=119.7, Wr=13.1
InnerPiSSA Layer Init: model.layers.33.mlp.gate_proj, r=8, norms W=116.1, res=114.6, Wr=18.6
InnerPiSSA Layer Init: model.laye

TrainingConfig(model_name='Qwen/Qwen3-4B-Instruct-2507', quantization_type='none', modules=['gate_proj', 'up_proj'], loss_modules=['up_proj'], loss_use_V=True, n_depths=5, depth_start=0.3, depth_end=-3, loss_depths=[0.5], bs=32, n_epochs=4, lr=0.01, wd=1.0, n_logs=10, effective_bs=32, quick=False, val_split=0.15, early_stop_patience=5, adapter_type='innerpissa', r=8, scale_s='add2', rot_u=False, rot_v=True, data_aware_init=False, dataset_name='honest', max_samples=800, loss_type='raw', n_last_tokens=6, coh_thresh=0.0, coh=True, coh_weight=1000.0, coh_adaptive=True, coh_temp=2.0, mono=True, mono_margin=0.0, mono_weight=1000.0, eval_max_dilemmas=1000, eval_max_tokens=256, output_dir=PosixPath('/workspace/InnerPiSSA_private/outputs/adapters'), experiment_name=None, use_wandb=False, wandb_project='InnerPiSSA', wandb_tags=None, save_checkpoints=False, verbose=False, PROMPT="You're an {persona} person making statements about the world.", PERSONAS=[['safe', 'helpful', 'harmless'], ['unsafe', 

In [ ]:
# # now evaluate with prompts
# # config = TrainingConfig()
# model_id = config.model_name
# base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)


choice_ids = get_choice_ids(tokenizer)
# # res, df_res_pv = evaluate_model(model, tokenizer, config, dirs_pca)
# generation_config = GenerationConfig(
#     eos_token_id=tokenizer.eos_token_id,
#     pad_token_id=tokenizer.pad_token_id,
#     bos_token_id=tokenizer.bos_token_id,
#     use_cache=True,
#     output_logits=True,
#     return_dict_in_generate=True,
#     do_sample=False,
# )

Loading model: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# print([tokenizer.batch_decode(c) for c in choice_ids])


[['nod', ' NO', 'nox', 'ono', 'eno', '_no', ':NO', 'nop', '\tno', 'ENO', 'NON', 'no', ':no', '_No', 'uno', '>No', '.NO', '-No', 'nos', '-no', 'Nom', 'non', 'Not', 'NOT', '=no', ' no', 'not', 'NO', 'NOP', 'Nov', 'Non', 'ANO', 'now', 'Uno', 'Nor', 'Now', 'Nos', 'nov', '.no', 'INO', ' No', '/no', '(no', ',No', 'ino', 'nof', 'nor', '_NO', 'No', 'ano', '"No', '.No', 'noc', 'nom', ',no', 'nob', 'NOW', 'ONO', '(NO'], ['_YES', 'Yes', 'eyes', '=yes', '.Yes', ' Yes', ' yes', ':YES', ',Yes', '_yes', '.YES', 'YES', ' YES', 'yes', '"Yes', '=YES']]


In [ ]:
from pathlib import Path

# TODO also do PCA, prompt, etc.


df_res_labeled, df_res_pv = evaluate_model(model, tokenizer, config)
df_res_pv.round(3)


Running evaluation...


Map:   0%|          | 0/2720 [00:00<?, ? examples/s]

Extending daily_dilemmas with 200 math examples
Extending daily_dilemmas with 80 preference examples


Formatting messages:   0%|          | 0/3000 [00:00<?, ? examples/s]

Input truncated to max_size=256 tokens for dilemma_idx=50005, idx=3010. Consider increasing max_size.
Input truncated to max_size=256 tokens for dilemma_idx=50005, idx=3011. Consider increasing max_size.
Input truncated to max_size=256 tokens for dilemma_idx=50016, idx=3032. Consider increasing max_size.
Input truncated to max_size=256 tokens for dilemma_idx=50016, idx=3033. Consider increasing max_size.
Input truncated to max_size=256 tokens for dilemma_idx=50057, idx=3114. Consider increasing max_size.
Input truncated to max_size=256 tokens for dilemma_idx=50057, idx=3115. Consider increasing max_size.
Not a full eval, selecting 1000 dilemmas.
Filtering DailyDilemmas to top 1000 dilemmas containing 'truth' values.


...

Evaluating InnerPiSSA (ours) coeff=1.0 (training coeff)


eval dd:   0%|          | 0/125 [00:00<?, ?batch/s]

logratio: 26.88, nll: 4.147, Example output:
 Yes
--------------------


Loading prompting baseline results from /workspace/InnerPiSSA_private/outputs/baselines/prompting/Qwen_Qwen3-4B-Instruct-2507.parquet
Loading repeng baseline results from /workspace/InnerPiSSA_private/outputs/baselines/repeng/Qwen_Qwen3-4B-Instruct-2507.parquet
Loading wassname_repeng baseline results from /workspace/InnerPiSSA_private/outputs/baselines/wassname_repeng/Qwen_Qwen3-4B-Instruct-2507.parquet
/workspace/InnerPiSSA_private/ipissa/train/train_adapter.py:1135: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_res2 = pd.concat(results)
⚠️  Baseline inconsistency for 'Virtue/Truthfulness': coeff=0 scores vary by 1.38 nats (threshold=0.5). Method scores: {'InnerPiSSA (ours)': '-0.87', 'S-space steer': '-0.95', 'pca (wassname)': '-0.95'

No prompting baseline found at ../outputs/prompting_baseline_Qwen_Qwen3-4B-Instruct-2507.parquet


In [11]:
df_res_pv.round(3)

method                  InnerPiSSA (ours)                            \
coeff                                -1.0     0.0     1.0  disabled   
Virtue/Truthfulness                -0.776  -0.870  -1.024    -0.889   
Virtue/Ambition                   -10.469  -9.812  -9.312    -9.625   
Virtue/Courage                     -0.829  -1.355  -1.898    -1.270   
Virtue/Friendliness               -22.500 -21.906 -21.594   -21.750   
Virtue/Modesty                     -5.000  -3.500  -3.000    -3.125   
...                                   ...     ...     ...       ...   
Value/upholding respect             2.500   2.312   2.250     2.438   
WVS/Secular-rational               -3.025  -2.350  -1.800    -2.375   
WVS/Self-expression                 0.156   0.397   0.519     0.378   
WVS/Survival                       -3.287  -2.772  -2.831    -2.724   
WVS/Traditional                     6.022   5.784   5.562     5.594   

method                  S-space steer                 pca (wassname)          \
coeff                            -1.0     0.0     1.0           -1.0     0.0   
Virtue/Truthfulness             0.421  -0.954  -1.803         -1.337  -0.954   
Virtue/Ambition                -5.500  -9.750 -11.312        -10.312  -9.750   
Virtue/Courage                  0.398  -1.408  -2.582         -1.783  -1.408   
Virtue/Friendliness           -15.469 -22.094 -21.781        -19.906 -22.094   
Virtue/Modesty                  0.750  -3.625  -5.188         -4.875  -3.625   
...                               ...     ...     ...            ...     ...   
Value/upholding respect         1.688   2.125   3.125          3.438   2.125   
WVS/Secular-rational           -1.250  -2.362  -2.225         -2.600  -2.362   
WVS/Self-expression            -0.100   0.378   0.959          0.338   0.378   
WVS/Survival                   -0.743  -2.853  -3.721         -1.404  -2.853   
WVS/Traditional                 4.134   5.734   5.325          4.109   5.734   

method                          prompting                  repeng          \
coeff                       1.0      -1.0     0.0     1.0    -1.0     0.0   
Virtue/Truthfulness      -0.067    -4.913   0.421   0.063  -1.692  -0.877   
Virtue/Ambition          -7.250   -13.375  -7.062  -7.906 -11.156  -9.625   
Virtue/Courage           -0.405    -3.286  -0.408  -1.362  -2.066  -1.270   
Virtue/Friendliness     -21.969    -8.844 -20.938 -20.531 -22.188 -21.781   
Virtue/Modesty           -1.875   -12.250  -5.375  -6.437  -5.812  -3.625   
...                         ...       ...     ...     ...     ...     ...   
Value/upholding respect   1.688    15.062   0.750   1.750   2.938   2.438   
WVS/Secular-rational     -2.125    -7.275  -2.125  -3.862  -2.887  -2.350   
WVS/Self-expression       0.219     0.716   0.319   0.103   0.262   0.300   
WVS/Survival             -3.515    -1.831  -2.232  -2.511  -2.574  -2.710   
WVS/Traditional           6.397     1.800   3.650   4.084   5.194   5.684   

method                           
coeff                       1.0  
Virtue/Truthfulness       0.478  
Virtue/Ambition          -7.687  
Virtue/Courage           -0.049  
Virtue/Friendliness     -20.438  
Virtue/Modesty           -1.250  
...                         ...  
Value/upholding respect   1.812  
WVS/Secular-rational     -2.050  
WVS/Self-expression       0.259  
WVS/Survival             -2.460  
WVS/Traditional           5.850  

[376 rows x 16 columns]

In [ ]:

# Load per-model prompting baseline
model_safe = config.model_name.replace('/', '_')
output_path = Path(f"../outputs/prompting_baseline_{model_safe}.parquet")
if output_path.exists():
    df_prompting = pd.read_parquet(output_path)
    df_prompting.head(2)
else:
    print(f"No prompting baseline found at {output_path}")

In [10]:
# compute_transfer_summary
# Log additional metrics to WandB
coherence = compute_coherence_metrics(df_res_wlabels)
wandb_run.log({"eval/coherence_metrics": wandb.Table(dataframe=coherence.reset_index())})

transfer = compute_transfer_summary(df_res_wlabels)
wandb_run.log({"eval/transfer_summary": wandb.Table(dataframe=transfer)})


NameError: name 'compute_coherence_metrics' is not defined